In [74]:
#This is a variant of "taketwo-dsmvp-v1" for the Jigsaw data.

In [75]:
#It is also a variant of the version that was pushed to the main branch of TakeTwo repo in Feb 21. 

In [76]:
#This is a version of "v1" demo notebook.

In [77]:
#CODE DEPENDENCY: sklearn, numpy, pandas, random, string. It uses from sklearn NB (Naive Bayes) and SGD (Support Vector Machine)

In [78]:
#DATA DEPENDENCY: In this version the "Jigsaw unintended biased in toxicity" data is used for demo purpose. 

In [79]:
#See https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/data?select=train.csv

In [80]:
#for detailed description of the data and instructions to download them. 

In [81]:
#As of now we are using only the "train.csv" portion of the data, which needs to be in the calling directory. 

In [82]:
from sklearn.datasets import fetch_20newsgroups

In [83]:
from sklearn.feature_extraction.text import CountVectorizer

In [84]:
from sklearn.linear_model import SGDClassifier

In [85]:
from sklearn.naive_bayes import MultinomialNB

In [86]:
from sklearn.pipeline import Pipeline

In [87]:
from sklearn.feature_extraction.text import TfidfTransformer

In [88]:
from sklearn.model_selection import GridSearchCV

In [89]:
import numpy as np

In [90]:
import pandas as pd

In [91]:
import random

In [92]:
import string

In [93]:
#Fetch the Jigsaw data and store them in appropriate arrays

In [94]:
jigsawTrain = pd.read_csv('train.csv')

In [95]:
jigsawTrain.shape

(1804874, 45)

In [96]:
allTrain = pd.DataFrame(jigsawTrain, columns = ['id', 'comment_text', 'severe_toxicity'])

In [97]:
allTrain.shape

(1804874, 3)

In [98]:
allTrain1M = allTrain.head(1000000)

In [99]:
allTrain1M.shape

(1000000, 3)

In [100]:
#allTrain = allTrain1M

In [101]:
blackData = jigsawTrain[(jigsawTrain.black == 1.0)]

In [102]:
allTrain.shape

(1804874, 3)

In [103]:
blackTrain = pd.DataFrame(blackData, columns = ['id', 'comment_text', 'severe_toxicity', 'black'])

In [104]:
blackTrain.shape

(7656, 4)

In [105]:
blackTrainId = blackTrain['id']

In [106]:
blackTrainText = blackTrain['comment_text']

In [107]:
blackTrainSevereToxicity = blackTrain.loc[:,'severe_toxicity']

In [108]:
blackTrainText.head(1)

516    BJ, your suggestion that low income residents ...
Name: comment_text, dtype: object

In [109]:
allTrainId = allTrain['id']

In [110]:
allTrainText = allTrain['comment_text']

In [111]:
allTrainSevereToxicity = allTrain.loc[:,'severe_toxicity']

In [112]:
allTrainSevereToxicity.head()

0    0.000000
1    0.000000
2    0.000000
3    0.000000
4    0.021277
Name: severe_toxicity, dtype: float64

In [113]:
blackTrain.head(5)

,id,comment_text,severe_toxicity,black
516,240627,"BJ, your suggestion that low income residents ...",0.0,1.0
1428,242346,"Well, this wasn't meant to be a dissertation. ...",0.0,1.0
1527,242454,So Hillary's appeal is to Democrats who make o...,0.0,1.0
1569,242498,Is the real point of this article to show that...,0.0,1.0
1623,242562,> A less sensational headline would be “Black ...,0.0,1.0


In [114]:
allTrain.head(5)

,id,comment_text,severe_toxicity
0,59848,"This is so cool. It's like, 'would you want yo...",0.000000
1,59849,Thank you!! This would make my life a lot less...,0.000000
2,59852,This is such an urgent design problem; kudos t...,0.000000
3,59855,Is this something I'll be able to install on m...,0.000000
4,59856,haha you guys are a bunch of losers.,0.021277


In [115]:
blackTrainId.iloc[0]

240627

In [116]:
blackTrainText.iloc[0]

"BJ, your suggestion that low income residents of Portland do not understand their own needs and desires perplexes me.  It is a bit snobby- as if the poor need middle class whites to tell them what they should want. And, for blacks and latinos, home ownership[ is pretty much their only chance for a nest egg.  Unless they are union they don't really get the jobs with stock options and generous 401k matches.  The fact that redlining once existed doesn't make the handful of black residents around Alberta less attached to their homes.\n\nYou are struggling with he fact that the working class folks in Portland do not want to live the New Urbanist dream.  Look at the big study Metro did to gauge housing preferences. \n\nFinally, I note you put a lot of trust in the people who own large apartment complexes to not exploit the inherent vulnerability that comes with being a renter.  The SFH gives people autonomy.  Yes, housing prices can drop, but so can any other investment.  Take my 401k- plea

In [117]:
blackTrainSevereToxicity.iloc[0]

0.0

In [118]:
len(blackTrain)

7656

In [119]:
categories = ["non-toxic", "toxic"]

In [120]:
from collections import namedtuple
data_set = namedtuple('data_set', ["data", "urls", "target_names", "target"])([],[],categories,[])

In [121]:
def flipCoin(bias):
    f = random.random()
    return True if f<bias else False

In [122]:
for i in range(0,10):
    print(flipCoin(0.25))

False
True
False
False
False
True
True
False
False
True


In [123]:
for x in range(len(blackTrain)):
    if (blackTrainSevereToxicity.iloc[x] > 0):
        data_set.target.append(1)
        data_set.urls.append(blackTrainId.iloc[x])
        data_set.data.append(blackTrainText.iloc[x])
    elif flipCoin(0.25):
        data_set.target.append(0)
        data_set.urls.append(blackTrainId.iloc[x])
        data_set.data.append(blackTrainText.iloc[x])

In [124]:
from collections import namedtuple
all_data_set = namedtuple('data_set', ["data", "urls", "target_names", "target"])([],[],categories,[])

In [125]:
from collections import namedtuple
all_train_set = namedtuple('data_set', ["data", "urls", "target_names", "target"])([],[],categories,[])

In [126]:
from collections import namedtuple
all_test_set = namedtuple('data_set', ["data", "urls", "target_names", "target"])([],[],categories,[])

In [127]:
downSampling = 0.06

In [128]:
for x in range(len(allTrain)):
    if (allTrainSevereToxicity.iloc[x] > 0):
        all_data_set.target.append(1)
        all_data_set.urls.append(allTrainId.iloc[x])
        all_data_set.data.append(allTrainText.iloc[x])
    elif flipCoin(downSampling):
        all_data_set.target.append(0)
        all_data_set.urls.append(allTrainId.iloc[x])
        all_data_set.data.append(allTrainText.iloc[x])

In [129]:
#Here is a version in which training and test data are separately generated from all data

In [130]:
trainRatio = 0.9

In [131]:
for x in range(len(allTrain)):
    if (allTrainSevereToxicity.iloc[x] > 0):
        if flipCoin(trainRatio): 
            all_train_set.target.append(1)
            all_train_set.urls.append(allTrainId.iloc[x])
            all_train_set.data.append(allTrainText.iloc[x])
        else: 
            all_test_set.target.append(1)
            all_test_set.urls.append(allTrainId.iloc[x])
            all_test_set.data.append(allTrainText.iloc[x])
    elif flipCoin(downSampling):
        if flipCoin(trainRatio): 
            all_train_set.target.append(0)
            all_train_set.urls.append(allTrainId.iloc[x])
            all_train_set.data.append(allTrainText.iloc[x])
        else: 
            all_test_set.target.append(0)
            all_test_set.urls.append(allTrainId.iloc[x])
            all_test_set.data.append(allTrainText.iloc[x])

In [132]:
#either leave data_set as is (generated from blackData) or set it to all_data_set (generated from all the data)

In [133]:
data_set = all_test_set

In [134]:
len(data_set.target)

20830

In [135]:
 data_set.target[0:10]

[1, 1, 1, 1, 1, 1, 0, 0, 1, 0]

In [136]:
sum(data_set.target)

10714

In [137]:
data_set.urls[0:10]

[59861, 239607, 239986, 240424, 240546, 240598, 240781, 240782, 240879, 241315]

In [138]:
data_set.data[0:2]

['hahahahahahahahhha suck it.',
 "Yet call out all Muslims for the acts of a few will get you pilloried.   So why is it okay to smear an entire religion over these few idiots?  Or is this because it's okay to bash Christian sects?"]

In [139]:
marker_train = all_train_set

In [140]:
#Here a separate test data set should be used.

In [141]:
marker_test = all_test_set

In [142]:
for t in marker_train.target[:10]:
...     print(marker_train.target_names[t])

toxic
toxic
toxic
toxic
non-toxic
non-toxic
non-toxic
toxic
non-toxic
non-toxic


In [143]:
#marker_train.data is an array of strings (containing as many "texts" as there are coming from Marker.)

In [144]:
#marker_train.target is an array of integers (containing the class indices for the above texts' class labels)

In [145]:
#marker_train_target_names is an array of class label names indexed by the array indices.

In [146]:
len(marker_train.target)

186506

In [147]:
sum(marker_train.target)

93922

In [148]:
len(marker_test.data)

20830

In [149]:
for t in marker_train.target[:10]:
...     print(marker_train.target_names[t])

toxic
toxic
toxic
toxic
non-toxic
non-toxic
non-toxic
toxic
non-toxic
non-toxic


In [150]:
for t in marker_test.target[:10]:
...     print(marker_train.target_names[t])

toxic
toxic
toxic
toxic
toxic
toxic
non-toxic
non-toxic
toxic
non-toxic


In [151]:
#Classes for model representations packaging the components in the modeling pipeline.
#For SGDClassifier and NaiveBayes.
#We may want to extend it to utilize "Pipeline()" as in "ModelRepSGD0", which is not in use at the moment.

In [152]:
class ModelRepSGD:
    def __init__(self, count_vect, X_train_counts, tf_transformer, text_clf):
        self.count_vect = CountVectorizer()
        self.X_train_counts = self.count_vect.fit_transform(marker_train.data)
        self.tf_transformer = TfidfTransformer(use_idf=False).fit(self.X_train_counts)
        self.text_clf = Pipeline([
            ('vect', CountVectorizer()),
            ('tfidf', TfidfTransformer()),
            ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                  alpha=1e-3, random_state=42,
                                  max_iter=5, tol=None)),
        ])

In [153]:
class ModelRepNB:
    def __init__(self, count_vect, X_train_counts, tf_transformer, clf):
        self.count_vect = CountVectorizer()
        self.X_train_counts = self.count_vect.fit_transform(marker_train.data)
        self.tf_transformer = TfidfTransformer(use_idf=False).fit(self.X_train_counts)
        self.clf = MultinomialNB()

In [154]:
#Functions for model training - for SGDClassifier and NaiveBayes (SGD one is untested)

In [155]:
def model_train_SGD(marker_train): 
    count_vect = CountVectorizer()
    X_train_counts = count_vect.fit_transform(marker_train.data)
    tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
    X_train_tf = tf_transformer.transform(X_train_counts)
    text_clf = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', SGDClassifier(loss='hinge', penalty='l2',
                              alpha=1e-3, random_state=42,
                              max_iter=5, tol=None)),
    ])
    text_clf.fit(marker_train.data, marker_train.target)
    trained_model = ModelRepSGD(count_vect, X_train_counts, tf_transformer, text_clf)
    trained_model.text_clf = text_clf
    return trained_model

In [156]:
def model_train_NB(marker_train): 
    count_vect = CountVectorizer()
    X_train_counts = count_vect.fit_transform(marker_train.data)
    tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
    X_train_tf = tf_transformer.transform(X_train_counts)
    clf = MultinomialNB().fit(X_train_tf, marker_train.target)
    trained_model = ModelRepNB(count_vect, X_train_counts, tf_transformer, clf)
    trained_model.clf = clf
    return trained_model

In [157]:
#Function for model testing (common for SGDClassifier and NB)

In [158]:
def model_test(flagger_new_test, trained_model):
    count_vect = trained_model.count_vect
    tf_transformer = trained_model.tf_transformer
    clf = trained_model.clf
    X_new_counts = count_vect.transform(flagger_new_test)
    X_new_tfidf = tf_transformer.transform(X_new_counts)
    predicted = clf.predict(X_new_tfidf)
    return predicted

In [159]:
#Now testing model_train and model_test

In [160]:
trained_model_NB = model_train_NB(marker_train)

In [161]:
trained_model_SGD = model_train_SGD(marker_train)

In [162]:
#Inspecting the components of the trained model

In [224]:
type(trained_model_NB)

__main__.ModelRepNB

In [164]:
trained_model_NB.count_vect

CountVectorizer()

In [165]:
trained_model_NB.tf_transformer

TfidfTransformer(use_idf=False)

In [166]:
trained_model_NB.clf

MultinomialNB()

In [167]:
trained_model_SGD.count_vect

CountVectorizer()

In [168]:
trained_model_SGD.tf_transformer

TfidfTransformer(use_idf=False)

In [169]:
trained_model_SGD.text_clf

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf',
                 SGDClassifier(alpha=0.001, max_iter=5, random_state=42,
                               tol=None))])

In [170]:
trained_model_SGD.count_vect

CountVectorizer()

In [171]:
trained_model_SGD.tf_transformer

TfidfTransformer(use_idf=False)

In [172]:
#Calling model tester/predictor on a single new text from Flagger

In [173]:
#flagger_new_text = fetch_20newsgroups(subset='test',
#                                      categories=categories, shuffle=True, random_state=42)

In [174]:
flagger_new_text = ['haha you guys are a bunch of losers.']

In [175]:
predicted_NB = model_test(flagger_new_text, trained_model_NB)

In [176]:
predicted_SGD = trained_model_SGD.text_clf.predict(flagger_new_text)

In [177]:
predicted_NB

array([1])

In [178]:
predicted_SGD

array([1])

In [179]:
#Now evaluating NB and SGD models on a test data set

In [180]:
predicted_test_NB = model_test(marker_test.data, trained_model_NB)

In [181]:
predicted_test_SGD = trained_model_SGD.text_clf.predict(marker_test.data)

In [182]:
predicted_test_SGD.shape

(20830,)

In [183]:
sum(predicted_test_SGD)

8851

In [184]:
sum(predicted_test_NB)

11114

In [185]:
predicted_test_SGD[0:20]

array([1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1])

In [186]:
marker_test.target

[1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,


In [187]:
np.mean(predicted_test_NB == marker_test.target)

0.7622659625540087

In [189]:
np.mean(predicted_test_SGD == marker_test.target)

0.7121939510321651

In [273]:
!pip install -U ibm_watson_machine_learning
!pip install -U ibm-cos-sdk-core


Requirement already up-to-date: ibm_watson_machine_learning in /Users/upkar/opt/anaconda3/lib/python3.8/site-packages (1.0.100)
Processing /Users/upkar/Library/Caches/pip/wheels/54/a5/95/ac2068077b0f41719c2a69a03fdfbdd76ad02cdbb0984ad1a7/ibm_cos_sdk_core-2.7.0-py2.py3-none-any.whl
  Attempting uninstall: ibm-cos-sdk-core
    Found existing installation: ibm-cos-sdk-core 2.10.0
    Uninstalling ibm-cos-sdk-core-2.10.0:
      Successfully uninstalled ibm-cos-sdk-core-2.10.0
Processing /Users/upkar/Library/Caches/pip/wheels/ce/df/1e/1da07def7155e0dd5cb2d93925ac6c0026042ecd3d3411ed9e/ibm_cos_sdk_core-2.10.0-py2.py3-none-any.whl
ERROR: ibm-cos-sdk 2.7.0 has requirement ibm-cos-sdk-core==2.7.0, but you'll have ibm-cos-sdk-core 2.10.0 which is incompatible.
ERROR: ibm-cos-sdk-s3transfer 2.7.0 has requirement ibm-cos-sdk-core==2.7.0, but you'll have ibm-cos-sdk-core 2.10.0 which is incompatible.
  Attempting uninstall: ibm-cos-sdk-core
    Found existing installation: ibm-cos-sdk-core 2.7.0
  

In [274]:
from ibm_watson_machine_learning import APIClient

In [276]:
# add apikey here
wml_credentials = {
                   "url": "https://eu-gb.ml.cloud.ibm.com",
                   "apikey":"insert api key here"
                  }

client = APIClient(wml_credentials)

In [278]:
space_metadata = {
    'name': 'taketwo-wml-space',
    'description': 'taketwo-wml-space',
}

In [279]:
#obtained this from WML resource on IBM Cloud
space_id = '2d3e8ea7-efa2-4efd-ae26-8a46e7a714df'

In [280]:
client.spaces.list(limit=10)

------------------------------------  -----------------  ------------------------
ID                                    NAME               CREATED
2d3e8ea7-efa2-4efd-ae26-8a46e7a714df  taketwo-wml-space  2021-07-16T00:54:38.389Z
------------------------------------  -----------------  ------------------------


In [281]:
client.set.default_space(space_id)

'SUCCESS'

In [282]:
sofware_spec_uid = client.software_specifications.get_id_by_name("default_py3.7")

In [283]:
metadata = {
            client.repository.ModelMetaNames.NAME: 'Take Two Scikit model',
            client.repository.ModelMetaNames.TYPE: 'scikit-learn_0.23',
            client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid
}

published_model = client.repository.store_model(
    model=trained_model_NB.clf,
    meta_props=metadata,
    training_data=None,
    training_target=None)


In [284]:
import json

published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

{
  "entity": {
    "software_spec": {
      "id": "e4429883-c883-42b6-87a8-f419d64088cd",
      "name": "default_py3.7"
    },
    "type": "scikit-learn_0.23"
  },
  "metadata": {
    "created_at": "2021-07-16T16:57:10.046Z",
    "id": "d195ce8b-182f-44e1-a424-817bef2896a4",
    "modified_at": "2021-07-16T16:57:15.264Z",
    "name": "Take Two Scikit model",
    "owner": "IBMid-270006CE7D",
    "space_id": "2d3e8ea7-efa2-4efd-ae26-8a46e7a714df"
  },
  "system": {
    "warnings": []
  }
}


In [285]:
models_details = client.repository.list_models()

------------------------------------  ---------------------  ------------------------  -----------------
ID                                    NAME                   CREATED                   TYPE
d195ce8b-182f-44e1-a424-817bef2896a4  Take Two Scikit model  2021-07-16T16:57:10.002Z  scikit-learn_0.23
569f90bf-a5f0-4f9b-9d8c-89a881493c1f  Take Two Scikit model  2021-07-16T16:41:17.002Z  scikit-learn_0.23
032f80f3-75a0-4a74-95eb-e5994465cc3f  Take Two Scikit model  2021-07-16T02:44:40.002Z  scikit-learn_0.23
------------------------------------  ---------------------  ------------------------  -----------------


In [286]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "take-two-deployment",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

created_deployment = client.deployments.create(published_model_uid, meta_props=metadata)



#######################################################################################

Synchronous deployment creation for uid: 'd195ce8b-182f-44e1-a424-817bef2896a4' started

#######################################################################################


initializing
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='af0cf9ba-97ba-470c-982e-665259df01bd'
------------------------------------------------------------------------------------------------




In [287]:
deployment_uid = client.deployments.get_uid(created_deployment)
deployment_uid

'af0cf9ba-97ba-470c-982e-665259df01bd'

In [288]:
scoring_endpoint = client.deployments.get_scoring_href(created_deployment)
print(scoring_endpoint)

https://eu-gb.ml.cloud.ibm.com/ml/v4/deployments/af0cf9ba-97ba-470c-982e-665259df01bd/predictions


In [289]:
client.deployments.list()


------------------------------------  -------------------  -----  ------------------------
GUID                                  NAME                 STATE  CREATED
af0cf9ba-97ba-470c-982e-665259df01bd  take-two-deployment  ready  2021-07-16T16:57:29.769Z
11633376-4d22-4934-a7ec-6e5cb3a23293  take-two-deployment  ready  2021-07-16T02:45:45.117Z
------------------------------------  -------------------  -----  ------------------------


In [249]:
client.deployments.get_details(deployment_uid)


{'entity': {'asset': {'id': '032f80f3-75a0-4a74-95eb-e5994465cc3f'},
  'custom': {},
  'deployed_asset_type': 'model',
  'hardware_spec': {'id': 'Not_Applicable', 'name': 'S', 'num_nodes': 1},
  'name': 'take-two-deployment',
  'online': {},
  'space_id': '2d3e8ea7-efa2-4efd-ae26-8a46e7a714df',
  'status': {'online_url': {'url': 'https://eu-gb.ml.cloud.ibm.com/ml/v4/deployments/11633376-4d22-4934-a7ec-6e5cb3a23293/predictions'},
   'state': 'ready'}},
 'metadata': {'created_at': '2021-07-16T02:45:45.117Z',
  'id': '11633376-4d22-4934-a7ec-6e5cb3a23293',
  'modified_at': '2021-07-16T02:45:45.117Z',
  'name': 'take-two-deployment',
  'owner': 'IBMid-270006CE7D',
  'space_id': '2d3e8ea7-efa2-4efd-ae26-8a46e7a714df'}}

In [268]:
# trying to create input format that is needed for WML
count_vect = trained_model_NB.count_vect
tf_transformer = trained_model_NB.tf_transformer
clf = trained_model_NB.clf
X_new_counts = count_vect.transform(marker_test.data)
X_new_tfidf = tf_transformer.transform(X_new_counts)
# predicted = clf.predict(X_new_tfidf)

In [269]:
X_new_tfidf

<20830x99873 sparse matrix of type '<class 'numpy.float64'>'
	with 768324 stored elements in Compressed Sparse Row format>

In [ ]:
# prediction with WML

import requests

# NOTE: you must manually set API_KEY below using information retrieved from your IBM Cloud account.
API_KEY = wml_credentials['api_key']
token_response = requests.post('https://iam.cloud.ibm.com/identity/token', data={"apikey": API_KEY, "grant_type": 'urn:ibm:params:oauth:grant-type:apikey'})
mltoken = token_response.json()["access_token"]

header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

# NOTE: manually define and pass the array(s) of values to be scored in the next line
payload_scoring = {"input_data": [{"fields": [array_of_input_fields], "values": [array_of_values_to_be_scored, another_array_of_values_to_be_scored]}]}

response_scoring = requests.post('https://eu-gb.ml.cloud.ibm.com/ml/v4/deployments/11633376-4d22-4934-a7ec-6e5cb3a23293/predictions?version=2021-07-16?version=2021-07-16', json=payload_scoring, headers={'Authorization': 'Bearer ' + mltoken})
print("Scoring response")
print(response_scoring.json())

In [250]:
# this command shows the input format required by the deployed model
client.deployments.ScoringMetaNames.show()

---------------------  ----  --------  ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
META_PROP NAME         TYPE  REQUIRED  SCHEMA
NAME                   str   N
INPUT_DATA             list  N         [{'name(optional)': 'string', 'id(optional)': 'string', 'fields(optional)': 'array[string]', 'values': 'array[array[string]]'}]
INPUT_DATA_REFERENCES  list  N         [{'id(optional)': 'string', 'name(optional)': 'string', 'type(required)': 'string', 'connection(required)': {'href(required)': 'string'}, 'location(required)': {'bucket': 'string', 'path': 'string'}, 'schema(optional)': {'id(required)': 'string', 'fields(required)': [{'name(required)': 'string', 'type(required)': 's